In [1]:
# load libraries 
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

In [2]:
# set seeds
torch.manual_seed(2020)
np.random.seed(2020)

In [3]:
# load data
train = np.loadtxt("ECG200_TRAIN.txt")
test  = np.loadtxt("ECG200_TEST.txt")

# the first column is label
X_train = train[:, 1:]       
y_train = train[:, 0].astype(int)     
X_test  = test[:, 1:]              
y_test  = test[:, 0].astype(int)

# original label is -1, +1, we convert to 0, 1 for BCE
y_train = (y_train == 1).astype(np.float32)
y_test  = (y_test == 1).astype(np.float32)


# convert to torch tensor
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)  # (N,1)
X_test  = torch.tensor(X_test, dtype=torch.float32)
y_test  = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)


print("X_train:", X_train.shape, "y_train:", y_train.shape)

X_train: torch.Size([100, 96]) y_train: torch.Size([100, 1])


In [4]:
# define a baseline model of logistic regression

class LogisticECG(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(96, 1) 

    def forward(self, x):
        logits = self.linear(x)
        prob = torch.sigmoid(logits)        
        return prob                       

# not in use in this step
# model = LogisticECG()

In [5]:
# define an advanced model of MLP with 1 hidden layer

class MLPECG(nn.Module):
    def __init__(self, hidden_dim):    
        super().__init__()
        self.fc1 = nn.Linear(96, hidden_dim)   
        self.act = nn.ReLU()                   
        self.fc2 = nn.Linear(hidden_dim, 1)    

    def forward(self, x):
        h = self.fc1(x)                     
        h = self.act(h)                       
        logits = self.fc2(h)                    
        prob = torch.sigmoid(logits)           
        return prob

In [6]:
# Full-batch 
num_epochs = 200

learning_rate = 0.01

hidden_dims = [16, 32, 64]

results = {}

for hidden_dim in hidden_dims:
    print("Training MLP with hidden_dim =", hidden_dim)

    model = MLPECG(hidden_dim=hidden_dim)

    num_params = sum(p.numel() for p in model.parameters())
    print("Number of parameters:", num_params)

    loss_function = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

    loss_history = []
    acc_history = []

    for epoch in range(num_epochs):
        model.train()

        optimizer.zero_grad()
        y_prob = model(X_train)
        loss = loss_function(y_prob, y_train)
        loss.backward()
        optimizer.step()

        with torch.no_grad():
            y_prob_new = model(X_train)
            train_pred = (y_prob_new >= 0.5).float()
            train_acc = (train_pred == y_train).float().mean().item()

        loss_history.append(loss.item())
        acc_history.append(train_acc)

    # evaluate
    model.eval()
    with torch.no_grad():
        test_prob = model(X_test)
        test_pred = (test_prob >= 0.5).float()
        test_acc = (test_pred == y_test).float().mean().item()

    print("Test Accuracy:", round(test_acc, 3))
    results[hidden_dim] = test_acc


Training MLP with hidden_dim = 16
Number of parameters: 1569
Test Accuracy: 0.85
Training MLP with hidden_dim = 32
Number of parameters: 3137
Test Accuracy: 0.89
Training MLP with hidden_dim = 64
Number of parameters: 6273
Test Accuracy: 0.87


In [7]:
hidden_dim = 32
learning_rates = [0.001, 0.01, 0.1]

results_lr = {}

for lr in learning_rates:
    print("Training MLP with hidden_dim =", hidden_dim, "and lr =", lr)

    model = MLPECG(hidden_dim=hidden_dim)

    loss_function = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()

        y_prob = model(X_train)
        loss = loss_function(y_prob, y_train)

        loss.backward()
        optimizer.step()

    # evaluate
    model.eval()
    with torch.no_grad():
        test_prob = model(X_test)
        test_pred = (test_prob >= 0.5).float()
        test_acc = (test_pred == y_test).float().mean().item()

    print("Test Accuracy:", round(test_acc, 3))
    results_lr[lr] = test_acc


Training MLP with hidden_dim = 32 and lr = 0.001
Test Accuracy: 0.86
Training MLP with hidden_dim = 32 and lr = 0.01
Test Accuracy: 0.88
Training MLP with hidden_dim = 32 and lr = 0.1
Test Accuracy: 0.87
